In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# ========== python ==========
import os
from pathlib import Path
from logging import Logger
from typing import List, Dict, Tuple, Optional, Union, Callable, Final, Literal, get_args
from operator import itemgetter, attrgetter
import itertools
from IPython.display import display

from utils.setup import setup_logger, get_device
from const.const_values import PROJECT_DIR

os.chdir(PROJECT_DIR)
logger: Logger = setup_logger(__name__, f'{PROJECT_DIR}/log/jupyter_run.log')
device = get_device(device_name='cpu', logger=logger)

In [2]:
# jupyter
import seaborn as sns
import matplotlib.pyplot as plt
# Machine learning
import numpy as np
import pandas as pd
import h5py
import optuna
# torch
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter
# torch ignite
from ignite.engine import Engine
from ignite.handlers import Checkpoint
# My items
from models.datasets.data_helper import MyDataHelper, MyDataLoaderHelper, DefaultTokens
from models.datasets.datasets_for_story import StoryTriple
# My utils
from utils.setup import load_param
from utils.torch import load_model, torch_fix_seed
# main function
from run_for_KGC import main_function


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [4]:
from const.const_values import CPU, MODEL
from models.KGModel.kg_model import HEAD, RELATION, TAIL
from utils.torch_ignite import TRAINER, EVALUATOR
from const.const_values import DATASETS, DATA_HELPER, DATA_LOADERS, TRAIN_RETURNS

In [5]:
SEED: Final[int] = 42
args_path = f'{PROJECT_DIR}/models/kgc03/SpeckledBand075/param.pkl'
model_path = f'{PROJECT_DIR}/models/kgc03/SpeckledBand075/model.pth'

In [6]:
args = load_param(args_path)

# args.pre_train = True
args.logger = logger
args.device = device
args.batch_size = 1
args.pre_train=False
args.init_embedding_using_bert = False
args.model_path = model_path
args.only_load_trainer_evaluator = True

del args.optuna_file, args.device_name, args.pid, args.study_name, args.n_trials
logger.info(args)

2023-02-04 05:42:16 - INFO - 929291086.py - 13 - Namespace(notebook=False, console_level='info', logfile='models/kgc03/SpeckledBand075/log.log', param_file='models/kgc03/SpeckledBand075/param.pkl', train_anyway=False, old_data=0, tensorboard_dir='models/kgc03/SpeckledBand075/tensorboard', checkpoint_dir='models/kgc03/SpeckledBand075/checkpoint/', model_path='/Users/ryoyakaneda/Documents/学校/M1Study/knowledge_graph/models/kgc03/SpeckledBand075/model.pth', resume_from_checkpoint=False, resume_from_last_point=False, only_load_trainer_evaluator=True, resume_checkpoint_path=None, pre_train=False, train_valid_test=False, only_train=True, use_for_challenge100=False, use_for_challenge090=False, use_for_challenge075=True, use_title='SpeckledBand', do_optuna=False, story_special_num=5, relation_special_num=5, entity_special_num=5, padding_token_e=0, cls_token_e=1, mask_token_e=2, sep_token_e=3, bos_token_e=4, padding_token_r=0, cls_token_r=1, mask_token_r=2, sep_token_r=3, bos_token_r=4, padding_

In [7]:
torch_fix_seed(seed=SEED)
return_dict = main_function(args, logger=logger)

2023-02-04 05:42:16 - INFO - run_for_KGC.py - 893 - ----- make datahelper start. -----
2023-02-04 05:42:16 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-04 05:42:16 - INFO - data_helper.py - 334 - entity num: 7812
2023-02-04 05:42:16 - INFO - data_helper.py - 335 - relation num: 62
2023-02-04 05:42:16 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-04 05:42:16 - INFO - data_helper.py - 607 - ========== Show DataHelper ==========
2023-02-04 05:42:16 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-04 05:42:16 - INFO - data_helper.py - 334 - entity num: 7812
2023-02-04 05:42:16 - INFO - data_helper.py - 335 - relation num: 62
2023-02-04 05:42:16 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-04 05:42:16 - INFO - data_helper.py - 609 - entity_special_dicts: {0: '<pad_e>', 1: '<cls_e>', 2: '<mask_e>', 3: '<sep_e>', 4: '<bos_e>'}
2023-02-04 05:42:16 - INFO - data_helper.py - 610 - relation_special_dicts: {

In [8]:
model = return_dict[MODEL]

dataset_train: StoryTriple = return_dict[DATASETS][0]
triple: torch.Tensor = dataset_train.triple
data_helper: MyDataHelper = return_dict[DATA_HELPER]
evaluator: Checkpoint = return_dict[TRAIN_RETURNS][EVALUATOR]

load_model(model, args.model_path, device)
model.eval()

entities, relations = data_helper.processed_entities, data_helper.processed_relations
d_e, d_r = {e: i for i, e in enumerate(entities)}, {r: i for i, r in enumerate(relations)}

triple_df = pd.DataFrame([(entities[_t[0]], relations[_t[1]], entities[_t[2]]) for _t in triple], columns=[HEAD, RELATION, TAIL])
story_entities = triple_df[HEAD].tolist()

In [9]:
triple_df

,head,relation,tail
0,<bos_e>,<bos_r>,<bos_e>
1,AbbeyGrange:001,kgc:subject,AllTitle:Holmes
2,AbbeyGrange:001,kgc:hasPredicate,word.predicate:wakeUp
3,AbbeyGrange:001,kgc:whom,AllTitle:Watson
4,AbbeyGrange:001,kgc:infoSource,AllTitle:Watson
...,...,...,...
17303,AbbeyGrange:084,kgc:subject,AbbeyGrange:Theresa
17304,AbbeyGrange:084,kgc:hasPredicate,word.predicate:rest
17305,AbbeyGrange:084,kgc:where,AbbeyGrange:room_of_Lady_Brackenstall
17306,<bos_e>,<bos_r>,<bos_e>


In [10]:
# This is sub functions for getting Attention.
def extract(_model, target, inputs):
    """This is sub functions for getting Attention.

    """
    features = None

    def forward_hook(_module, _inputs, _):
        nonlocal features
        x, _, _ = _inputs
        outputs = _module.forward(x, x, x, need_weights=True)[1]
        features = outputs.detach().clone()

    handle = target.register_forward_hook(forward_hook)

    _model.eval()
    _model(inputs, torch.LongTensor([[]]), torch.LongTensor([[]]), torch.LongTensor([[]]))

    handle.remove()

    return features

# This is main functions for getting Attention.
def get_attention(input_):
    """This is main functions for getting Attention.

    """
    assert len(input_) == 1
    features = extract(model, model.transformer.layers[-1].self_attn, input_)[0]
    df_attention = pd.DataFrame([[entities[h], relations[r], entities[t]]+[features[j, i].item() for j in range(len(features))] for i, (h, r, t) in enumerate(input_[0])])
    df_attention.columns=[HEAD, RELATION, TAIL] + [f'atten_from{i}' for i in range(len(df_attention.columns)-3)]
    return df_attention

def show_attention_heatmap(df_attention):
    sns.heatmap(df_attention.iloc[:,3:])
    plt.show()

In [11]:
MASK_E = DefaultTokens.MASK_E
KILL = 'word.predicate:kill'

TAKE = 'word.predicate:take'
BRING = 'word.predicate:bring'
DIE = 'word.predicate:die'
HIDE = 'word.predicate:hide'

In [12]:
bos_triple = [d_e[DefaultTokens.BOS_E], d_r[DefaultTokens.BOS_R],d_e[DefaultTokens.BOS_E]]
mask_e_id = d_e[DefaultTokens.MASK_E]
Holmes_id =d_e['AllTitle:Holmes']

def make_ranking(from_story_name, to_story_name, predicate_, whom_, subject_, why_, what_, where_):
    if not (from_story_name is None and to_story_name is None):
        _start_index = story_entities.index(from_story_name)-1
        _end_index = len(story_entities) - story_entities[::-1].index(to_story_name)
    else:
        _start_index = 0
        _end_index = 0
    question_ = torch.tensor(
        [
            # [... last 80 scenes ...],
            # ...
            bos_triple,
            [mask_e_id, d_r['kgc:infoSource'],     Holmes_id      ],
            [mask_e_id, d_r['kgc:hasPredicate'],   d_e[predicate_]],
            [mask_e_id, d_r['kgc:whom'],           d_e[whom_     ]],
            [mask_e_id, d_r['kgc:subject'],        d_e[subject_  ]],
            [mask_e_id, d_r['kgc:why'],            d_e[why_      ]],
            [mask_e_id, d_r['kgc:what'],           d_e[what_     ]],
            [mask_e_id, d_r['kgc:where'],          d_e[where_    ]],
        ]
    )
    mask_ = torch.zeros_like(question_, dtype=torch.bool) # not mask all position
    mask_[1:, 0] = True                                   # where head position without bos token
    mask_[1:, 2] = True                                   # where tail position without bos token

    last_triples = triple[_start_index: _end_index]

    questions = torch.cat([last_triples, question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros_like(last_triples), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    data_list = []
    with torch.no_grad():
        _, (story_pred, relation_pred, entity_pred) = model(questions, masks[:,0], masks[:,1], masks[:,2])
        sorted_ = torch.argsort(entity_pred, dim=1, descending=True)
        for i in range(sorted_.shape[1]):
            ans_= sorted_[:, i]
            info_source_, predicate_pred, whom_pred, subject_pred, why_pred, what_pred, where_pred = ans_
            data_list.append([entities[predicate_pred], entities[whom_pred], entities[subject_pred], entities[why_pred], entities[what_pred], entities[where_pred]])
    df_ranking = pd.DataFrame(data_list, columns=['predicate', 'whom', 'subject', 'why', 'what', 'where'])
    df_attension = get_attention(questions)

    return df_ranking, df_attension

In [13]:
def main_func01(_title, _victim_name, criminal, predicate, _last_index, _story_len):
    from_ = f'{_title}:{_last_index-_story_len+1}'
    to_ = f'{_title}:{_last_index}'
    predicate = predicate
    victim = f'{_title}:{_victim_name}'
    criminal = f'{_title}:{criminal}'
    df_ranking, df_attension = make_ranking(
        from_, to_, predicate, victim, MASK_E, MASK_E, MASK_E, MASK_E)
    df_ranking.index.name='ranking'

    pred_rank = df_ranking.index[df_ranking['subject']==criminal].tolist()
    pred_rank = pred_rank[0] if len(pred_rank)==1 else -1
    logger.info(f"The pred ranking about {criminal} is {pred_rank}")
    display(df_ranking.iloc[:max(20, pred_rank)])
    len_ = len(df_attension)
    for i in range(len_-10, len_):
        print(f"index={i}, triple={df_attension.iloc[i,:3].tolist()}, attention list")
        display(df_attension.sort_values(f'atten_from{i}', ascending=False).iloc[:,[0,1,2,3+i]],)
    return df_ranking, df_attension

def check_killer(_title, _victim_name, _killer_name, _last_index, _story_len):
    return main_func01(_title, _victim_name, _killer_name, KILL, _last_index, _story_len)

In [14]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

### SpeckledBand(まだらの紐)
Who killed Julia? (criminal & explanation)
被害者: Julia
犯人: Roylott
犯行に用いたもの: snake
犯行動機: 母の相続財産を独占したい

In [15]:
def do_SpeckledBand_pred():
    title = 'SpeckledBand'
    victim_name = 'Julia'
    killer_name = 'Roylott'
    last_index = 300
    story_len = 80

    df_ranking, df_attention = check_killer(title, victim_name, killer_name, last_index, story_len)
    return df_ranking, df_attention
do_SpeckledBand_pred()
pass

2023-02-04 05:42:18 - INFO - 3696358453.py - 13 - The pred ranking about SpeckledBand:Roylott is 3


,predicate,whom,subject,why,what,where
ranking,,,,,,
0,word.predicate:go,AllTitle:Watson,AllTitle:Watson,SpeckledBand:1883-04-02T00,AllTitle:Holmes,SpeckledBand:Julia_s_bedroom
1,word.predicate:send,AllTitle:Holmes,AllTitle:Holmes,CrookedMan:Barclay,AllTitle:Watson,ACaseOfIdentity:Sutherland
2,AllTitle:Holmes,AbbeyGrange:Lady_Brackenstall,AbbeyGrange:Stanley_Hopkins,AbbeyGrange:Jack_Croker,SpeckledBand:Helen,AllTitle:Holmes
3,word.predicate:enter,AbbeyGrange:Stanley_Hopkins,SpeckledBand:Roylott,SilverBlaze:Fitzroy_Simpson,DevilsFoot:Sterndale,SpeckledBand:Roylott_s_bedroom
4,word.predicate:kill,AbbeyGrange:Criminal,DevilsFoot:Mortimer,DateTime:1883-04-02T00:00:00,ResidentPatient:Elderly_man,word.predicate:examine
5,word.predicate:think,SpeckledBand:Julia,AbbeyGrange:Lady_Brackenstall,AllTitle:Holmes,AbbeyGrange:Lady_Brackenstall,DateTime:1883-04-01T15:00:00
6,CrookedMan:Barclay,ACaseOfIdentity:Hosmer,word.predicate:examine,SpeckledBand:Julia,DancingMen:Local_doctor,ACaseOfIdentity:Windibank
7,word.predicate:want,SpeckledBand:Roylott,SpeckledBand:Helen,SpeckledBand:Helen,SpeckledBand:Roylott_s_bedroom,AbbeyGrange:Stanley_Hopkins
8,word.predicate:sit,SpeckledBand:1883-04-02T00,word.predicate:have,CrookedMan:Rasheen,AbbeyGrange:Stanley_Hopkins,CrookedMan:Nancy


index=498, triple=['SpeckledBand:300', 'kgc:when', 'SpeckledBand:1883-04-02T01'], attention list


,head,relation,tail,atten_from498
415,SpeckledBand:287,kgc:when,SpeckledBand:1881-12-02T00,0.020614
409,SpeckledBand:286,kgc:when,SpeckledBand:1881-12-02T00,0.015310
342,SpeckledBand:275,kgc:infoSource,SpeckledBand:Helen,0.011391
384,SpeckledBand:282,kgc:when,SpeckledBand:1881-12-02T00,0.009759
222,SpeckledBand:258,kgc:what,SpeckledBand:261,0.009296
...,...,...,...,...
9,SpeckledBand:222,kgc:hasPredicate,word.predicate:letEnter,0.000237
262,SpeckledBand:262,kgc:hasPredicate,word.predicate:go,0.000220
40,SpeckledBand:227,kgc:hasPredicate,word.predicate:exist,0.000210
172,SpeckledBand:250,kgc:hasPredicate,word.predicate:notSatisfied,0.000182


index=499, triple=['SpeckledBand:300', 'kgc:time', 'DateTime:1883-04-02T01:00:00'], attention list


...

,head,relation,tail,atten_from505
17,SpeckledBand:223,kgc:when,SpeckledBand:1883-04-01T15,0.006849
222,SpeckledBand:258,kgc:what,SpeckledBand:261,0.006753
376,SpeckledBand:281,kgc:subject,SpeckledBand:rope_of_bell,0.006685
342,SpeckledBand:275,kgc:infoSource,SpeckledBand:Helen,0.006495
224,SpeckledBand:258,kgc:what,SpeckledBand:263,0.006431
...,...,...,...,...
282,SpeckledBand:265,kgc:hasPredicate,word.predicate:go,0.000240
156,SpeckledBand:247,kgc:hasPredicate,word.predicate:exist,0.000237
9,SpeckledBand:222,kgc:hasPredicate,word.predicate:letEnter,0.000224
137,SpeckledBand:244,kgc:hasPredicate,word.predicate:takeDown,0.000221


index=506, triple=['<mask_e>', 'kgc:what', '<mask_e>'], attention list


,head,relation,tail,atten_from506
117,SpeckledBand:241,kgc:what,SpeckledBand:simple_chair,0.009596
16,SpeckledBand:223,kgc:infoSource,SpeckledBand:Helen,0.008749
224,SpeckledBand:258,kgc:what,SpeckledBand:263,0.008298
116,SpeckledBand:241,kgc:what,SpeckledBand:elbow_chair,0.008151
118,SpeckledBand:241,kgc:what,SpeckledBand:round_table,0.007800
...,...,...,...,...
150,SpeckledBand:246,kgc:hasPredicate,word.predicate:notExist,0.000187
156,SpeckledBand:247,kgc:hasPredicate,word.predicate:exist,0.000181
196,SpeckledBand:254,kgc:hasPredicate,word.predicate:exist,0.000178
9,SpeckledBand:222,kgc:hasPredicate,word.predicate:letEnter,0.000133


index=507, triple=['<mask_e>', 'kgc:where', '<mask_e>'], attention list


,head,relation,tail,atten_from507
16,SpeckledBand:223,kgc:infoSource,SpeckledBand:Helen,0.010408
342,SpeckledBand:275,kgc:infoSource,SpeckledBand:Helen,0.009616
418,SpeckledBand:289,kgc:subject,SpeckledBand:Helen,0.008318
326,SpeckledBand:272,kgc:subject,SpeckledBand:VentilationHole,0.006301
29,SpeckledBand:225,kgc:infoSource,SpeckledBand:Helen,0.006232
...,...,...,...,...
2,SpeckledBand:221,kgc:hasPredicate,word.predicate:exist,0.000151
196,SpeckledBand:254,kgc:hasPredicate,word.predicate:exist,0.000148
156,SpeckledBand:247,kgc:hasPredicate,word.predicate:exist,0.000145
137,SpeckledBand:244,kgc:hasPredicate,word.predicate:takeDown,0.000139
